# Design and implement a deep neural network for regression using the PyTorch. Define the model architecture, compile the model, and prepare it for training.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.targets[index]
# 1. Load dataset (replace with your actual dataset path)
def load_dataset():
    np.random.seed(42)
    data = np.random.randn(1000, 5)  # 1000 samples, 5 features
    target = 2 * data[:, 0] + 3 * data[:, 1] - data[:, 2] + np.random.randn(1000) * 0.5  # Synthetic target
    return pd.DataFrame(data, columns=[f"feature_{i}" for i in range(5)]), target
# 2. Handle missing values
def handle_missing_values(df):
    df = df.fillna(df.mean())
    return df
# 3. Normalize data
def normalize_data(df, target):
    scaler = StandardScaler()
    df_normalized = scaler.fit_transform(df)
    target_normalized = (target - np.mean(target)) / np.std(target)
    return df_normalized, target_normalized
# 4. Define the Deep Neural Network model
class DeepNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(DeepNNModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out
# 5. Training function
def train_model(model, dataloader, criterion, optimizer, epochs=100):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for features, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}/{epochs}, Loss: {running_loss/len(dataloader):.4f}')
# 6. Example usage
if __name__ == "__main__":
    # Load and preprocess dataset
    df, target = load_dataset()
    df.iloc[::10, 0] = np.nan  # Introduce some missing values for demonstration
    df = handle_missing_values(df)
    features, targets = normalize_data(df, target)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

    # Create DataLoader
    train_dataset = CustomDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    # Initialize the model, loss function, and optimizer
    input_dim = X_train.shape[1]
    hidden_dim1 = 64
    hidden_dim2 = 32
    output_dim = 1
    model = DeepNNModel(input_dim, hidden_dim1, hidden_dim2, output_dim)
    criterion = nn.MSELoss()  # Mean Squared Error for regression
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    # Train the model
    train_model(model, train_loader, criterion, optimizer, epochs=100)
    # Evaluate the model on test data
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
        predictions = model(X_test_tensor)
        test_loss = criterion(predictions, y_test_tensor)
        print(f'Test Loss: {test_loss.item():.4f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0/100, Loss: 1.0097
Epoch 10/100, Loss: 0.9790
Epoch 20/100, Loss: 0.9777
Epoch 30/100, Loss: 0.9785
Epoch 40/100, Loss: 0.9781
Epoch 50/100, Loss: 0.9782
Epoch 60/100, Loss: 0.9784
Epoch 70/100, Loss: 0.9780
Epoch 80/100, Loss: 0.9787
Epoch 90/100, Loss: 0.9780
Test Loss: 1.0880


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
